In [1]:
from TopoAnalysis import dem as d

In [2]:
elevation = d.Elevation.load('rapel_SRTMGL130m_dem_utm.tif')
print(elevation._georef_info.xllcenter, elevation._georef_info.yllcenter)

125032.2717442584 6046054.072526088


In [3]:
import os

print(f"Current working directory: {os.getcwd()}")
print(f"Is file accessible? {os.path.isfile('rapel_SRTMGL130m_dem_utm.tif')}")

Current working directory: /Users/Glong1/Desktop/Andes/AndesTG/Rapel
Is file accessible? True


In [ ]:
# Set the working directory to where your file is located
#os.chdir('AndesTG/Rapel/rapel_SRTMGL130m_dem_utm.tif')

# Verify the working directory
#print(f"New working directory: {os.getcwd()}")

In [4]:
filled = d.FilledElevation(elevation = elevation)

In [5]:
flow_direction = d.FlowDirectionD8(flooded_dem = filled)
area = d.Area(flow_direction = flow_direction)
log_area = d.LogArea(area = area)

In [6]:
filled.save('rapel_dem_utm_filled30m')
flow_direction.save('rapel_fd_utm30m')
area.save('rapel_area_utm30m')
log_area.save('rapel_logarea_utm30m')

In [ ]:
"""RP-1	river sand	-33.908524	-71.811668
RP-3	river sand	-34.292129	-71.432546
RP-4	river sand	-34.301552	-71.45435
RP-5	river sand	-34.381495	-71.241344
RP-7	river sand	-34.120319	-71.377098
RP-8	river sand	-34.675155	-70.880005
RP-9	river sand	-34.435514	-70.784442
RP-10	river sand	-34.248991	-70.560314
RP-S2	river sand	-33.9458637	-71.7458024
RP-S3	river sand	-33.96972222	-71.70333333
RP-S3u	river sand	-34.05527778	-71.70305556
RP-S4u	river sand	-34.1531149	-71.5784708
"""

In [ ]:
## Convert Lat long to easting northing
from pyproj import Transformer

def lat_long_to_utm_epsg32719(lat, lon):
    """
    Converts latitude and longitude to UTM easting and northing using EPSG: . . ..

    Parameters:
        lat (float): Latitude in decimal degrees.
        lon (float): Longitude in decimal degrees.

    Returns:
        tuple: (easting, northing) in meters.
    """
    # Define the transformer for EPSG:4326 (WGS 84) to EPSG:32719 (WGS 84 / UTM Zone 19S)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32719", always_xy=True)
    # Transform coordinates
    easting, northing = transformer.transform(lon, lat)
    return easting, northing

# Coords
coordinates = [
    (,),
]

# Perform conversion and print results
for lon, lat in coordinates:
    easting, northing = lat_long_to_utm_epsg32719(lat, lon)
    print(f"{easting:.2f}, {northing:.2f}")

In [ ]:
# these are 'new' points based on the 30m DEM now available (the correction is to the flow line based on
# where the samples were actually taken from 

In [10]:
samples = [['RP-1',	240012.3742,	6244480.719],
           ['RP-S2',	246249.2076,	6240448.447],
           ['RP-S3',	250212.9354,	6237948.848],
           ['RP-S3u',	250603.5211,	6228481.157],
           ['RP-S4u',	262281.9748,	6217862.614],
           ['RP-4',	274855.8121,	6202358.94],
           ['RP-3',	276089.43710,	6202762.02337],
           ['RP-7',	279864.4664,	6222354.853],
           ['RP-10',	356294.7321,	6209145.568],
           ['RP-9',	336045.0559,	6188091.153],
           ['RP-8',	327706.5081,	6161529.282],
           ['RP-5',	293923.0144,	6193146.699]
          ]

In [11]:
for sample, x, y in samples:
    try:
        basin_mask = d.Mask(flow_direction = flow_direction, outlets = ((x, y),))
        basin_mask.save(sample.replace('-','_') + '_mask')

        print(f"Finished processing sample: {sample}")

    except Exception as e: 
        print(f"failed to process sample {sample}: {e}")

Finished processing sample: RP-1
Finished processing sample: RP-S2
Finished processing sample: RP-S3
Finished processing sample: RP-S3u
Finished processing sample: RP-S4u
Finished processing sample: RP-4
Finished processing sample: RP-3
Finished processing sample: RP-7
Finished processing sample: RP-10
Finished processing sample: RP-9
Finished processing sample: RP-8
Finished processing sample: RP-5


In [12]:
os.chdir('/Users/Glong1/Desktop/Andes/AndesTG/Rapel/masks')

In [14]:
import numpy as np

for sample, _, _ in samples:
    basin_mask = d.Mask.load(sample.replace('-','_') + '_mask')
    total_area = np.sum(basin_mask._griddata)*np.power(basin_mask._georef_info.dx,2)
    print(sample, float(total_area) / 1.0E6, "km2")

RP-1 13760.997960900397 km2
RP-S2 33.99005374925599 km2
RP-S3 209.82974749592972 km2
RP-S3u 112.1653820442073 km2
RP-S4u 9.349920370877914 km2
RP-4 1217.7547541332438 km2
RP-3 3902.040665963297 km2
RP-7 966.0250354545303 km2
RP-10 2460.6201672799793 km2
RP-9 303.19953795371583 km2
RP-8 1838.9286342046253 km2
RP-5 6105.677535017025 km2


In [1]:
import os
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# List of masked raster files (replace with your actual file paths)
masked_rasters = ["RP_S2_mask", "RP_S3_mask", "RP_S3u_mask", "RP_S4u_mask", "RP_1_mask", "RP_3_mask", "RP_4_mask", "RP_5_mask", "RP_7_mask", "RP_8_mask", "RP_9_mask",
                  "RP_10_mask"]  # Add your files here

# Output folder for polygon shapefiles
output_folder = "output_polygons"
os.makedirs(output_folder, exist_ok=True)

# Specify the CRS (optional). Example: "EPSG:32719" for UTM Zone 19S
user_defined_crs = "EPSG:32719"  # Change or set to None to use the raster's CRS

# Loop through each masked raster and convert to polygon
for raster_file in masked_rasters:
    try:
        # Open the raster file
        with rasterio.open(raster_file) as src:
            # Read the raster data
            image = src.read(1)  # Read the first band
            mask = image > 0  # Only include non-zero pixels
            
            # Convert raster to polygons
            results = (
                {"properties": {"value": value}, "geometry": shape(geom)}
                for geom, value in shapes(image, mask=mask, transform=src.transform)
            )
            
            # Convert to a GeoDataFrame
            gdf = gpd.GeoDataFrame.from_features(results, crs=src.crs)
            
            # Override CRS if user_defined_crs is provided
            if user_defined_crs:
                gdf = gdf.set_crs(user_defined_crs, allow_override=True)
                print(f"CRS set to: {user_defined_crs}")
            
            # Save as a shapefile
            output_file = os.path.join(output_folder, os.path.basename(raster_file).replace(".tif", "_polygon.shp"))
            gdf.to_file(output_file)
            print(f"Successfully converted {raster_file} to {output_file}")
    
    except Exception as e:
        print(f"Failed to process {raster_file}: {e}")


CRS set to: EPSG:32719
Successfully converted RP_S2_mask to output_polygons/RP_S2_mask
CRS set to: EPSG:32719
Successfully converted RP_S3_mask to output_polygons/RP_S3_mask
CRS set to: EPSG:32719
Successfully converted RP_S3u_mask to output_polygons/RP_S3u_mask
CRS set to: EPSG:32719
Successfully converted RP_S4u_mask to output_polygons/RP_S4u_mask
CRS set to: EPSG:32719
Successfully converted RP_1_mask to output_polygons/RP_1_mask
CRS set to: EPSG:32719
Successfully converted RP_3_mask to output_polygons/RP_3_mask
CRS set to: EPSG:32719
Successfully converted RP_4_mask to output_polygons/RP_4_mask
CRS set to: EPSG:32719
Successfully converted RP_5_mask to output_polygons/RP_5_mask
CRS set to: EPSG:32719
Successfully converted RP_7_mask to output_polygons/RP_7_mask
CRS set to: EPSG:32719
Successfully converted RP_8_mask to output_polygons/RP_8_mask
CRS set to: EPSG:32719
Successfully converted RP_9_mask to output_polygons/RP_9_mask
CRS set to: EPSG:32719
Successfully converted RP_10_m